In [ ]:
# Packages
import pandas as pd
from scipy.io import arff
import pandas as pd
import numpy as np

URV                                                                            MESIIA

Neural and Evolutionary Computation (NEC)
Assignment 4: Optimization with Genetic Algorithm

Teachers: Dr. Jordi Duch, Dr. Sergio Gomez

Student: Natzaret Gálvez Rísquez